In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Set environment variables
import os

os.environ["SCAL_TYPE"] = "complex"
os.environ["PRECISION"] = "double"
os.environ["MY_NUMBA_TARGET"] = "cuda"
 
# Add cle_fun to PYTHON_PATH
import sys
sys.path.append("../../clonscal")

In [2]:
import numba, math # type: ignore 
from numba import cuda # type: ignore 
import numpy as np
from numba import float32
from src.numba_target import myjit


lattice_size = 2**20
threadsperblock = 256
blockspergrid = math.ceil(lattice_size / threadsperblock)

phi0 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
phi1 = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
eta = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
dS_norm = cuda.to_device(np.empty(lattice_size, dtype = np.float32))
mass_real = np.float64(1)
interaction = np.float64(1)
dt_ada = np.float64(0.1)

/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/__init__.py:3: UserWarning: The module `llvmlite.llvmpy` is deprecated and will be removed in the future.
  warnings.warn(
/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/core.py:8: UserWarning: The module `llvmlite.llvmpy.core` is deprecated and will be removed in the future. Equivalent functionality is provided by `llvmlite.ir`.
  warnings.warn(
/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/llvmlite/llvmpy/passes.py:17: UserWarning: The module `llvmlite.llvmpy.passes` is deprecated and will be removed in the future. If you are using this code, it should be inlined into your own project.
  warnings.warn(


Using CUDA


In [ ]:
d_array = cuda.to_device(np.random.random((int(1e7))))

In [8]:
def unzero(d_array):
    array = d_array.copy_to_host()
    return array[array!=0]
%timeit unzero(d_array)

17 ms ± 363 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [6]:
import numba
from numba import cuda
import numpy as np

@myjit
def count_uncorr_kernel(idx, array, threshold, uncorr_counter):
    if array[idx] > threshold:
        uncorr_counter += 1


In [72]:
import numba
from numba import cuda
import numpy as np

@cuda.jit
def mark_valid_elements(A, threshold, valid_mask):
    idx = cuda.grid(1)
    if idx < len(A):
        # Mark elements above threshold with 1, else 0
        valid_mask[idx] = 1 if A[idx] > threshold else 0

@cuda.jit
def gather_values(A, valid_mask, result, valid_count):
    idx = cuda.grid(1)
    if idx < len(A):
        if valid_mask[idx] == 1:
            # Atomically store the valid value in the result array
            index = cuda.atomic.add(valid_count, 0, 1)  # Atomic increment for valid_count
            result[index] = A[idx]

def filter_array(A, threshold):
    N = len(A)

    # Allocate memory for mask and result array
    valid_mask = np.zeros(N, dtype=np.int32)
    result = np.zeros(N, dtype=A.dtype)  # Result array for filtered values
    valid_count = np.zeros(1, dtype=np.int32)  # Tracks the number of valid elements

    # Copy data to device
    d_A = cuda.to_device(A)
    d_valid_mask = cuda.to_device(valid_mask)
    d_result = cuda.to_device(result)
    d_valid_count = cuda.to_device(valid_count)

    # Launch kernel to mark valid elements
    threads_per_block = 256
    blocks_per_grid = (N + (threads_per_block - 1)) // threads_per_block
    mark_valid_elements[blocks_per_grid, threads_per_block](d_A, threshold, d_valid_mask)

    # Launch kernel to gather values into the result array
    gather_values[blocks_per_grid, threads_per_block](d_A, d_valid_mask, d_result, d_valid_count)

    # After kernel execution, d_result contains the filtered values
    # The size of the valid elements is stored in d_valid_count

    return d_result, d_valid_count[0]  # Return result array and number of valid elements

# Example usage:
A = np.random.random(int(1e5))
threshold = 0.3
d_result, valid_count = filter_array(A, threshold)

# d_result contains the filtered values on the device, and valid_count is the number of valid elements
print(f"Filtered values (on device): {d_result.copy_to_host()[:valid_count]}")
print(f"Number of valid elements: {valid_count}")


Filtered values (on device): [0.30275189 0.42834024 0.75274812 ... 0.55465462 0.4757526  0.37490834]
Number of valid elements: 70125


In [16]:
@myjit
def test_cuda_kernel(idx, array, val):
    array[idx] += val

def test_numpy(array, val):
    array += val

In [8]:
array = np.empty(int(1e5))
d_array = cuda.to_device(array)
array.size

100000

In [ ]:


from src.numba_target import my_parallel_loop
size = array.size
my_parallel_loop(test_cuda_kernel, )

/home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/numba/cuda/compiler.py:726: NumbaPerformanceWarning: Grid size (1) < 2 * SM count (96) will likely result in GPU under utilization due to low occupancy.
  warn(NumbaPerformanceWarning(msg))


TypingError: Failed in cuda mode pipeline (step: nopython frontend)
[1m[1mNo implementation of function Function(<built-in function getitem>) found for signature:
 
 >>> getitem(array(int64, 0d, C), int32)
 
There are 22 candidate implementations:
[1m    - Of which 20 did not match due to:
    Overload of function 'getitem': File: <numerous>: Line N/A.
      With argument(s): '(array(int64, 0d, C), int32)':[0m
[1m     No match.[0m
[1m    - Of which 2 did not match due to:
    Overload in function 'GetItemBuffer.generic': File: numba/core/typing/arraydecl.py: Line 166.
      With argument(s): '(array(int64, 0d, C), int32)':[0m
[1m     Rejected as the implementation raised a specific error:
       NumbaTypeError: [1mcannot index array(int64, 0d, C) with 1 indices: [int32][0m[0m
  raised from /home/fhelmberger/miniconda3/envs/cle_cuda/lib/python3.10/site-packages/numba/core/typing/arraydecl.py:88
[0m
[0m[1mDuring: typing of intrinsic-call at /tmp/ipykernel_12153/1450863335.